In [229]:
# loading the consolidated training dataset:

import pandas as pd
import numpy as np 
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, IsolationForest
from sklearn.model_selection import GroupKFold, cross_val_score,cross_val_predict
from sklearn.metrics import f1_score, precision_score, confusion_matrix, classification_report, accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

%run ./feature_engineering_final.ipynb




In [230]:
training_df

,Case#,P1_mean,P1_std,P1_min,P1_max,P1_skew,P1_kurtosis,P1_cycle1_max,P1_cycle1_std,P1_cycle2_max,...,SV3,SV4,BP1,BP2,BP3,BP4,BP5,BP6,BP7,BV1
0,1,1.984867,0.308519,0.436248,4.228663,1.639029,18.385586,4.226256,0.308483,4.228431,...,100,100,No,No,No,No,No,No,No,No
1,2,1.984931,0.308624,0.436435,4.256737,1.669368,18.490675,4.254314,0.308591,4.256499,...,100,100,No,No,No,No,No,No,No,No
2,3,1.984999,0.308539,0.436793,4.295627,1.682806,18.656679,4.293190,0.308512,4.295373,...,100,100,No,No,No,No,No,No,No,No
3,4,1.984965,0.308392,0.437050,4.295622,1.661384,18.595380,4.293183,0.308366,4.295375,...,100,100,No,No,No,No,No,No,No,No
4,5,1.984987,0.308432,0.437011,4.281213,1.648009,18.448610,4.278777,0.308409,4.280965,...,100,100,No,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,173,1.984983,0.278327,0.543228,4.016765,1.869240,15.538411,4.014407,0.278784,4.016529,...,100,100,No,No,No,Yes,No,No,No,No
173,174,1.984349,0.278177,0.820108,4.063794,2.193004,18.755654,4.061469,0.278336,4.063541,...,100,100,No,No,No,No,Yes,No,No,No
174,175,1.984209,0.274191,0.522027,4.182883,1.751404,21.326472,4.180296,0.274008,4.182612,...,100,100,No,No,No,No,No,Yes,No,No
175,176,1.985246,0.271765,0.579673,4.103252,1.993451,21.082582,4.100687,0.271008,4.102971,...,100,100,No,No,No,No,No,No,Yes,No


In [231]:
test_features_df

,Case#,P1_mean,P1_std,P1_min,P1_max,P1_skew,P1_kurtosis,P1_cycle1_max,P1_cycle1_std,P1_cycle2_max,...,P7_kurtosis,P7_cycle1_max,P7_cycle1_std,P7_cycle2_max,P7_cycle2_std,P7_cycle3_max,P7_cycle3_std,P7_fft_max,P7_fft_low,P7_fft_high
0,178,1.984422,0.300675,0.561754,4.411926,1.955759,19.904115,4.409431,0.300305,4.411653,...,7.456282,4.957685,0.562416,4.959830,0.544475,4.960276,0.544206,2368.250239,3343.796289,3826.472739
1,179,1.984932,0.305923,0.453799,4.103312,1.546036,17.500193,4.100935,0.305777,4.103057,...,7.572686,5.082534,0.582644,5.085373,0.570766,5.085864,0.570734,2368.534290,3301.727662,3763.237779
2,180,1.984972,0.308424,0.436712,4.262760,1.661200,18.461282,4.260331,0.308394,4.262515,...,6.439843,4.997548,0.573276,5.001972,0.584786,5.000872,0.575337,2361.863225,3330.577159,4100.950659
3,181,1.985002,0.308406,0.442751,4.279393,1.703869,18.633265,4.276943,0.308373,4.279134,...,6.524109,4.990073,0.573094,4.993812,0.582290,4.994317,0.583452,2368.896272,3307.432559,3749.581176
4,182,1.984954,0.308387,0.437060,4.270896,1.645871,18.417790,4.268466,0.308359,4.270658,...,6.938116,4.995626,0.583251,4.998082,0.570672,4.998432,0.568732,2368.837234,3304.822395,3765.596832
5,183,1.984890,0.308375,0.436635,4.238379,1.621332,18.314742,4.235972,0.308345,4.238146,...,7.301251,5.013583,0.577871,5.016722,0.566410,5.017334,0.566291,2367.321720,3305.668812,3791.540989
6,184,2.976179,0.325922,1.411117,5.319466,1.017982,16.733550,5.315434,0.325189,5.319051,...,7.250227,5.721555,0.568949,5.726516,0.574190,5.727131,0.574302,3559.135926,5076.851847,2199.719358
7,185,1.984882,0.308438,0.436636,4.233074,1.626207,18.297275,4.230665,0.308406,4.232831,...,7.153080,5.008386,0.581632,5.010837,0.569793,5.011322,0.569440,2368.818641,3303.993883,3870.942233
8,186,1.984192,0.281480,0.525377,4.156134,1.828446,20.282872,4.154228,0.281304,4.155922,...,11.243099,5.656658,0.600185,5.666543,0.610994,5.667324,0.610785,2365.340871,3383.569228,4868.995948
9,187,1.984911,0.308471,0.436629,4.267636,1.655919,18.536173,4.265194,0.308440,4.267377,...,7.296816,5.010933,0.572459,5.015120,0.563769,5.015618,0.564056,2366.563250,3304.944610,3768.839666


In [244]:
# TASK 1: Binary classification into normal/abnormal

# A very low mean validation score is obtained for the RF trained on the physics informed feature engineered training dataset
# need to move to a diff ML model

x_features=training_df[["P1_mean","P1_std","P1_min","P1_max","P1_skew","P1_kurtosis"
                        ,"P2_mean","P2_std","P2_min","P2_max","P2_skew","P2_kurtosis"
                        ,"P3_mean","P3_std","P3_min","P3_max","P3_skew","P3_kurtosis"
                        ,"P4_mean","P4_std","P4_min","P4_max","P4_skew","P4_kurtosis"
                        ,"P5_mean","P5_std","P5_min","P5_max","P5_skew","P5_kurtosis"
                        ,"P6_mean","P6_std","P6_min","P6_max","P6_skew","P6_kurtosis"
                        ,"P7_mean","P7_std","P7_min","P7_max","P7_skew","P7_kurtosis","P1_cycle1_max","P1_cycle1_std","P1_cycle2_max","P1_cycle2_std","P1_cycle3_max","P1_cycle3_std","P1_fft_max","P1_fft_low","P1_fft_high"
                        ,"P2_cycle1_max","P2_cycle1_std","P2_cycle2_max","P2_cycle2_std","P2_cycle3_max","P2_cycle3_std","P2_fft_max","P2_fft_low","P2_fft_high"
                        ,"P3_cycle1_max","P3_cycle1_std","P3_cycle2_max","P3_cycle2_std","P3_cycle3_max","P3_cycle3_std","P3_fft_max","P3_fft_low","P3_fft_high"
                        ,"P4_cycle1_max","P4_cycle1_std","P4_cycle2_max","P4_cycle2_std","P4_cycle3_max","P4_cycle3_std","P4_fft_max","P4_fft_low","P4_fft_high"
                        ,"P5_cycle1_max","P5_cycle1_std","P5_cycle2_max","P5_cycle2_std","P5_cycle3_max","P5_cycle3_std","P5_fft_max","P5_fft_low","P5_fft_high"
                        ,"P6_cycle1_max","P6_cycle1_std","P6_cycle2_max","P6_cycle2_std","P6_cycle3_max","P6_cycle3_std","P6_fft_max","P6_fft_low","P6_fft_high"
                        ,"P7_cycle1_max","P7_cycle1_std","P7_cycle2_max","P7_cycle2_std","P7_cycle3_max","P7_cycle3_std","P7_fft_max","P7_fft_low","P7_fft_high"]]                     


# encoding the labels dataframe:
# the Normal condition is mapped to 0
# the fault condition is mapped to 1
convert_dict={"Normal":0,"Fault":1,"Anomaly":1}


y_labels = training_df["Condition"].replace(convert_dict).infer_objects(copy=False)


# for group k fold 

groups=training_df["Spacecraft#"]

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold, cross_val_score,cross_val_predict

# Say we use 100 decision trees
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# This ensures we strictly separate Spacecraft 1, 2, and 3
# This also helps us in counteracting the unknown spacecraft problem that will be encountered during testing
gkf = GroupKFold(n_splits=3)

# Calculate scores using CV, this inherently fits the data
scores = cross_val_score(clf, x_features, y_labels, groups=groups, cv=gkf)


y_pred_task1 = cross_val_predict(clf, x_features, y_labels, groups=groups, cv=gkf)
print(y_pred_task1)
print(len(y_pred_task1))

# Display the results
print(f"Scores for each fold: {scores}")
print(f"Accuracy for task 1: {np.mean(scores)}")


C:\Users\samar\AppData\Local\Temp\ipykernel_28320\831047397.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_labels = training_df["Condition"].replace(convert_dict).infer_objects(copy=False)


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 0 0 1 0 0 1 1 1 0 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
177
Scores for each fold: [0.40677966 0.59322034 0.40677966]
Accuracy for task 1: 0.4689265536723164


In [245]:
# TASK 1: Binary classification into normal/abnormal

# After much trial and error with XGBoost and random forests which both gave a low accuracy of around 50-60%, decided to shift to K-NN
# Also conducted hyperparameter tuning to compute the optimal k

x_features=training_df[["P1_mean","P1_std","P1_min","P1_max","P1_skew","P1_kurtosis"
                        ,"P2_mean","P2_std","P2_min","P2_max","P2_skew","P2_kurtosis"
                        ,"P3_mean","P3_std","P3_min","P3_max","P3_skew","P3_kurtosis"
                        ,"P4_mean","P4_std","P4_min","P4_max","P4_skew","P4_kurtosis"
                        ,"P5_mean","P5_std","P5_min","P5_max","P5_skew","P5_kurtosis"
                        ,"P6_mean","P6_std","P6_min","P6_max","P6_skew","P6_kurtosis"
                        ,"P7_mean","P7_std","P7_min","P7_max","P7_skew","P7_kurtosis","P1_cycle1_max","P1_cycle1_std","P1_cycle2_max","P1_cycle2_std","P1_cycle3_max","P1_cycle3_std","P1_fft_max","P1_fft_low","P1_fft_high"
                        ,"P2_cycle1_max","P2_cycle1_std","P2_cycle2_max","P2_cycle2_std","P2_cycle3_max","P2_cycle3_std","P2_fft_max","P2_fft_low","P2_fft_high"
                        ,"P3_cycle1_max","P3_cycle1_std","P3_cycle2_max","P3_cycle2_std","P3_cycle3_max","P3_cycle3_std","P3_fft_max","P3_fft_low","P3_fft_high"
                        ,"P4_cycle1_max","P4_cycle1_std","P4_cycle2_max","P4_cycle2_std","P4_cycle3_max","P4_cycle3_std","P4_fft_max","P4_fft_low","P4_fft_high"
                        ,"P5_cycle1_max","P5_cycle1_std","P5_cycle2_max","P5_cycle2_std","P5_cycle3_max","P5_cycle3_std","P5_fft_max","P5_fft_low","P5_fft_high"
                        ,"P6_cycle1_max","P6_cycle1_std","P6_cycle2_max","P6_cycle2_std","P6_cycle3_max","P6_cycle3_std","P6_fft_max","P6_fft_low","P6_fft_high"
                        ,"P7_cycle1_max","P7_cycle1_std","P7_cycle2_max","P7_cycle2_std","P7_cycle3_max","P7_cycle3_std","P7_fft_max","P7_fft_low","P7_fft_high"]]                     


# encoding the labels dataframe:
# the Normal condition is mapped to 0
# the fault condition is mapped to 1
convert_dict={"Normal":0,"Fault":1,"Anomaly":1}


y_labels = training_df["Condition"].replace(convert_dict).infer_objects(copy=False)


# for group k fold 

groups=training_df["Spacecraft#"]


# After performing hyperparameter tuning, optimal k=7 

knn_pipeline = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=7))
scores_knn = cross_val_score(knn_pipeline, x_features, y_labels, groups=groups, cv=gkf)

y_pred = cross_val_predict(knn_pipeline, x_features, y_labels, groups=groups, cv=gkf)

prec = precision_score(y_labels, y_pred, pos_label=1)
f1 = f1_score(y_labels, y_pred, pos_label=1)
cm = confusion_matrix(y_labels, y_pred)
    
print(f"Task 1 Accuracy: {np.mean(scores_knn):.4f}")
print(f"Task 1 Precision: {prec}")
print(f"Task 1 F1 score: {f1}")
print(f"Task 1 Confusion Matrix: {cm}")

# Much better accuracy compared to the baseline model which uses random forests
# Great precision, so no false positives
# A lot of abnormal data (34) classified as normal 

Task 1 Accuracy: 0.8079
Task 1 Precision: 1.0
Task 1 F1 score: 0.6909090909090909
Task 1 Confusion Matrix: [[105   0]
 [ 34  38]]


C:\Users\samar\AppData\Local\Temp\ipykernel_28320\4089584735.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_labels = training_df["Condition"].replace(convert_dict).infer_objects(copy=False)


In [233]:
# TASK 2

# Identification of type of fault using random forests:

abnormal_df = training_df[training_df["Condition"] != "Normal"]

x_abnormal = abnormal_df[["P1_mean","P1_std","P1_min","P1_max","P1_skew","P1_kurtosis"
                        ,"P2_mean","P2_std","P2_min","P2_max","P2_skew","P2_kurtosis"
                        ,"P3_mean","P3_std","P3_min","P3_max","P3_skew","P3_kurtosis"
                        ,"P4_mean","P4_std","P4_min","P4_max","P4_skew","P4_kurtosis"
                        ,"P5_mean","P5_std","P5_min","P5_max","P5_skew","P5_kurtosis"
                        ,"P6_mean","P6_std","P6_min","P6_max","P6_skew","P6_kurtosis"
                        ,"P7_mean","P7_std","P7_min","P7_max","P7_skew","P7_kurtosis","P1_cycle1_max","P1_cycle1_std","P1_cycle2_max","P1_cycle2_std","P1_cycle3_max","P1_cycle3_std","P1_fft_max","P1_fft_low","P1_fft_high"
                        ,"P2_cycle1_max","P2_cycle1_std","P2_cycle2_max","P2_cycle2_std","P2_cycle3_max","P2_cycle3_std","P2_fft_max","P2_fft_low","P2_fft_high"
                        ,"P3_cycle1_max","P3_cycle1_std","P3_cycle2_max","P3_cycle2_std","P3_cycle3_max","P3_cycle3_std","P3_fft_max","P3_fft_low","P3_fft_high"
                        ,"P4_cycle1_max","P4_cycle1_std","P4_cycle2_max","P4_cycle2_std","P4_cycle3_max","P4_cycle3_std","P4_fft_max","P4_fft_low","P4_fft_high"
                        ,"P5_cycle1_max","P5_cycle1_std","P5_cycle2_max","P5_cycle2_std","P5_cycle3_max","P5_cycle3_std","P5_fft_max","P5_fft_low","P5_fft_high"
                        ,"P6_cycle1_max","P6_cycle1_std","P6_cycle2_max","P6_cycle2_std","P6_cycle3_max","P6_cycle3_std","P6_fft_max","P6_fft_low","P6_fft_high"
                        ,"P7_cycle1_max","P7_cycle1_std","P7_cycle2_max","P7_cycle2_std","P7_cycle3_max","P7_cycle3_std","P7_fft_max","P7_fft_low","P7_fft_high"]].to_numpy()

y_abnormal = abnormal_df["Condition"].replace({"Anomaly": 0, "Fault": 1}).to_numpy()

groups_abnormal = abnormal_df["Spacecraft#"].to_numpy()

# Say we use 100 decision trees
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# This ensures we strictly separate Spacecraft 1, 2, and 3
# This also helps us in counteracting the unknown spacecraft problem that will be encountered during testing
gkf = GroupKFold(n_splits=3)

scores_task2 = cross_val_score(clf, x_abnormal, y_abnormal, groups=groups_abnormal, cv=gkf)

y_pred = cross_val_predict(clf, x_abnormal, y_abnormal, groups=groups_abnormal, cv=gkf)

prec = precision_score(y_abnormal, y_pred, pos_label=1)
f1 = f1_score(y_abnormal, y_pred, pos_label=1)
cm = confusion_matrix(y_abnormal, y_pred)
    
print(f"Task 2 Accuracy: {np.mean(scores_task2):.4f}")
print(f"Task 2 Precision: {prec}")
print(f"Task 2 F1 score: {f1}")
print(f"Task 2 Confusion Matrix: {cm}")

# all metrics look good, can move on to task 3



C:\Users\samar\AppData\Local\Temp\ipykernel_28320\4286327252.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_abnormal = abnormal_df["Condition"].replace({"Anomaly": 0, "Fault": 1}).to_numpy()


Task 2 Accuracy: 0.9722
Task 2 Precision: 1.0
Task 2 F1 score: 0.9787234042553191
Task 2 Confusion Matrix: [[24  0]
 [ 2 46]]


In [234]:
# TASK 3
# Need to find the location of bubble anomaly
# USing the same random forest classifier:

anomaly_df = training_df[training_df["Condition"] == "Anomaly"]

x_anomaly = anomaly_df[["P1_mean","P1_std","P1_min","P1_max","P1_skew","P1_kurtosis"
                        ,"P2_mean","P2_std","P2_min","P2_max","P2_skew","P2_kurtosis"
                        ,"P3_mean","P3_std","P3_min","P3_max","P3_skew","P3_kurtosis"
                        ,"P4_mean","P4_std","P4_min","P4_max","P4_skew","P4_kurtosis"
                        ,"P5_mean","P5_std","P5_min","P5_max","P5_skew","P5_kurtosis"
                        ,"P6_mean","P6_std","P6_min","P6_max","P6_skew","P6_kurtosis"
                        ,"P7_mean","P7_std","P7_min","P7_max","P7_skew","P7_kurtosis","P1_cycle1_max","P1_cycle1_std","P1_cycle2_max","P1_cycle2_std","P1_cycle3_max","P1_cycle3_std","P1_fft_max","P1_fft_low","P1_fft_high"
                        ,"P2_cycle1_max","P2_cycle1_std","P2_cycle2_max","P2_cycle2_std","P2_cycle3_max","P2_cycle3_std","P2_fft_max","P2_fft_low","P2_fft_high"
                        ,"P3_cycle1_max","P3_cycle1_std","P3_cycle2_max","P3_cycle2_std","P3_cycle3_max","P3_cycle3_std","P3_fft_max","P3_fft_low","P3_fft_high"
                        ,"P4_cycle1_max","P4_cycle1_std","P4_cycle2_max","P4_cycle2_std","P4_cycle3_max","P4_cycle3_std","P4_fft_max","P4_fft_low","P4_fft_high"
                        ,"P5_cycle1_max","P5_cycle1_std","P5_cycle2_max","P5_cycle2_std","P5_cycle3_max","P5_cycle3_std","P5_fft_max","P5_fft_low","P5_fft_high"
                        ,"P6_cycle1_max","P6_cycle1_std","P6_cycle2_max","P6_cycle2_std","P6_cycle3_max","P6_cycle3_std","P6_fft_max","P6_fft_low","P6_fft_high"
                        ,"P7_cycle1_max","P7_cycle1_std","P7_cycle2_max","P7_cycle2_std","P7_cycle3_max","P7_cycle3_std","P7_fft_max","P7_fft_low","P7_fft_high"]].to_numpy()

loc_cols = ['BP1', 'BP2', 'BP3', 'BP4', 'BP5', 'BP6', 'BP7', 'BV1']

y_bubble_text = anomaly_df[loc_cols].idxmax(axis=1)
y_anomaly = y_bubble_text.replace({"BP1": 0, "BP2": 1, "BP3": 2, "BP4": 3, "BP5": 4, "BP6": 5, "BP7": 6, "BV1": 7}).to_numpy()


groups_anomaly = anomaly_df["Spacecraft#"].to_numpy()

scores_task3 = cross_val_score(clf, x_anomaly, y_anomaly, groups=groups_anomaly, cv=gkf)

y_pred = cross_val_predict(clf, x_anomaly, y_anomaly, groups=groups_anomaly, cv=gkf)
   
print(f"Task 3 Accuracy: {np.mean(scores_task3):.4f}")

# Dip in accuracy may be due to Curse of Dimensionality, only 24 out of 177 datapoints in the training set are clssified as a bubble anomaly, with around  105 features


C:\Users\samar\AppData\Local\Temp\ipykernel_28320\3052385686.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_anomaly = y_bubble_text.replace({"BP1": 0, "BP2": 1, "BP3": 2, "BP4": 3, "BP5": 4, "BP6": 5, "BP7": 6, "BV1": 7}).to_numpy()


Task 3 Accuracy: 0.7500


In [235]:
# TASK 4
# Applying a similar logic as task 3:
# We use the same Random forest model for identification of location of solenoid fault:

fault_df = training_df[training_df["Condition"] == "Fault"]

x_fault = fault_df[["P1_mean","P1_std","P1_min","P1_max","P1_skew","P1_kurtosis"
                        ,"P2_mean","P2_std","P2_min","P2_max","P2_skew","P2_kurtosis"
                        ,"P3_mean","P3_std","P3_min","P3_max","P3_skew","P3_kurtosis"
                        ,"P4_mean","P4_std","P4_min","P4_max","P4_skew","P4_kurtosis"
                        ,"P5_mean","P5_std","P5_min","P5_max","P5_skew","P5_kurtosis"
                        ,"P6_mean","P6_std","P6_min","P6_max","P6_skew","P6_kurtosis"
                        ,"P7_mean","P7_std","P7_min","P7_max","P7_skew","P7_kurtosis","P1_cycle1_max","P1_cycle1_std","P1_cycle2_max","P1_cycle2_std","P1_cycle3_max","P1_cycle3_std","P1_fft_max","P1_fft_low","P1_fft_high"
                        ,"P2_cycle1_max","P2_cycle1_std","P2_cycle2_max","P2_cycle2_std","P2_cycle3_max","P2_cycle3_std","P2_fft_max","P2_fft_low","P2_fft_high"
                        ,"P3_cycle1_max","P3_cycle1_std","P3_cycle2_max","P3_cycle2_std","P3_cycle3_max","P3_cycle3_std","P3_fft_max","P3_fft_low","P3_fft_high"
                        ,"P4_cycle1_max","P4_cycle1_std","P4_cycle2_max","P4_cycle2_std","P4_cycle3_max","P4_cycle3_std","P4_fft_max","P4_fft_low","P4_fft_high"
                        ,"P5_cycle1_max","P5_cycle1_std","P5_cycle2_max","P5_cycle2_std","P5_cycle3_max","P5_cycle3_std","P5_fft_max","P5_fft_low","P5_fft_high"
                        ,"P6_cycle1_max","P6_cycle1_std","P6_cycle2_max","P6_cycle2_std","P6_cycle3_max","P6_cycle3_std","P6_fft_max","P6_fft_low","P6_fft_high"
                        ,"P7_cycle1_max","P7_cycle1_std","P7_cycle2_max","P7_cycle2_std","P7_cycle3_max","P7_cycle3_std","P7_fft_max","P7_fft_low","P7_fft_high"]].to_numpy()

loc_cols = ["SV1",	"SV2",	"SV3",	"SV4"]

y_fault_text = fault_df[loc_cols].idxmax(axis=1)
y_fault = y_fault_text.replace({"SV1": 0, "SV2": 1, "SV3": 2, "SV4": 3}).to_numpy()


groups_fault = fault_df["Spacecraft#"].to_numpy()

scores_task4 = cross_val_score(clf, x_fault, y_fault, groups=groups_fault, cv=gkf)

print(f"Task 4 Accuracy: {np.mean(scores_task4)}")

C:\Users\samar\AppData\Local\Temp\ipykernel_28320\55370934.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_fault = y_fault_text.replace({"SV1": 0, "SV2": 1, "SV3": 2, "SV4": 3}).to_numpy()


Task 4 Accuracy: 0.8333333333333334


In [236]:
# TASK 5
# Need to predict the opening ratios of the faulty solenoid valves
# Need to use regression as the predicted values will be of numeric (interval) type
# we use the random forest regressor for this and measure accuracy in terms of Mean Absolute Error (MAE)
# when all the columns are used, then the mae came to be 12.28% opening ratio
# removing the unnecessay columns like skew,kurt, time segmented, fft
# after this, mae has reduced to 11.43 %, a slight improvement from the baseline model

fault_df = training_df[training_df["Condition"] == "Fault"]

x_fault = fault_df[["P1_mean","P1_std","P1_min","P1_max"
                        ,"P2_mean","P2_std","P2_min","P2_max"
                        ,"P3_mean","P3_std","P3_min","P3_max"
                        ,"P4_mean","P4_std","P4_min","P4_max"
                        ,"P5_mean","P5_std","P5_min","P5_max"
                        ,"P6_mean","P6_std","P6_min","P6_max"
                        ,"P7_mean","P7_std","P7_min","P7_max"]].to_numpy()

fault_cols = ['SV1', 'SV2', 'SV3', 'SV4']
# the minimum of the opening ratios indicate that the valve is not completely open and a solenoid valve exists
# so to predict the opening ratio, we need to take the minimum value
y_ratio = fault_df[fault_cols].min(axis=1).to_numpy()


groups_ratio = fault_df["Spacecraft#"].to_numpy()

# 100 DTs for regression task
regr = RandomForestRegressor(n_estimators=100, random_state=42)

# Calculating MAE
scores_task5 = cross_val_score(regr, x_fault, y_ratio, groups=groups_ratio, cv=gkf, scoring='neg_mean_absolute_error')


print("Maximum Absolute Error for Task 5 in opening ratio %" ,abs(np.mean(scores_task5)))



Maximum Absolute Error for Task 5 in opening ratio % 11.4375


In [237]:
# All five tasks are done using group k fold, we we know that all these models 
# are sufficent to learn from all the training datapoints
# now need to train all the models on all the traning data
# we use the isolation forest to look for anomalies (unknown defects)

cols_to_drop = ['Case#', 'Spacecraft#', 'Condition', 'SV1', 'SV2', 'SV3', 'SV4', 'BP1', 'BP2', 'BP3', 'BP4', 'BP5', 'BP6', 'BP7', 'BV1']
feature_cols = [c for c in training_df.columns if c not in cols_to_drop]
X_train_full = training_df[feature_cols].to_numpy()

print("1. Training the Isolation Forest for identification of any outliers (unknowns)")

gatekeeper = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
gatekeeper.fit(X_train_full)



1. Training the Isolation Forest for identification of any outliers (unknowns)


IsolationForest(contamination=0.01, random_state=42)

In [238]:
print("2. Task 1 using KNN")
y_t1 = training_df["Condition"].replace({"Normal": 0, "Anomaly": 1, "Fault": 1}).to_numpy()
model_t1 = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=7))
model_t1.fit(X_train_full, y_t1)


2. Task 1 using KNN


C:\Users\samar\AppData\Local\Temp\ipykernel_28320\3985759543.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_t1 = training_df["Condition"].replace({"Normal": 0, "Anomaly": 1, "Fault": 1}).to_numpy()


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=7))])

In [239]:
print("3. Task 2 using RF")
mask_ab = training_df["Condition"] != "Normal"
X_ab = training_df.loc[mask_ab, feature_cols].to_numpy()
y_t2 = training_df.loc[mask_ab, "Condition"].replace({"Anomaly": 0, "Fault": 1}).to_numpy()
model_t2 = RandomForestClassifier(n_estimators=100, random_state=42).fit(X_ab, y_t2)
model_t2

3. Task 2 using RF


C:\Users\samar\AppData\Local\Temp\ipykernel_28320\484552112.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_t2 = training_df.loc[mask_ab, "Condition"].replace({"Anomaly": 0, "Fault": 1}).to_numpy()


RandomForestClassifier(random_state=42)

In [240]:
print("4. Task 3 using RF")
mask_bub = training_df["Condition"] == "Anomaly"
X_bub = training_df.loc[mask_bub, feature_cols].to_numpy()
loc_cols = ['BP1', 'BP2', 'BP3', 'BP4', 'BP5', 'BP6', 'BP7', 'BV1']
y_t3 = training_df.loc[mask_bub, loc_cols].idxmax(axis=1).replace({
    "BP1": 0, "BP2": 1, "BP3": 2, "BP4": 3, "BP5": 4, "BP6": 5, "BP7": 6, "BV1": 7
}).to_numpy()
model_t3 = RandomForestClassifier(n_estimators=100, random_state=42).fit(X_bub, y_t3)

4. Task 3 using RF


C:\Users\samar\AppData\Local\Temp\ipykernel_28320\2849831780.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_t3 = training_df.loc[mask_bub, loc_cols].idxmax(axis=1).replace({


In [241]:
print("4. Task 4 using RF and RFR")
mask_val = training_df["Condition"] == "Fault"
X_val = training_df.loc[mask_val, feature_cols].to_numpy()
sv_cols = ['SV1', 'SV2', 'SV3', 'SV4']
y_t4 = training_df.loc[mask_val, sv_cols].idxmin(axis=1).replace({"SV1": 0, "SV2": 1, "SV3": 2, "SV4": 3}).to_numpy()
y_t5 = training_df.loc[mask_val, sv_cols].min(axis=1).to_numpy()

model_t4 = RandomForestClassifier(n_estimators=100, random_state=42).fit(X_val, y_t4)
model_t5 = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_val, y_t5)

4. Task 4 using RF and RFR


C:\Users\samar\AppData\Local\Temp\ipykernel_28320\2681742841.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_t4 = training_df.loc[mask_val, sv_cols].idxmin(axis=1).replace({"SV1": 0, "SV2": 1, "SV3": 2, "SV4": 3}).to_numpy()


In [242]:
# This marks the end of training the models on the complete training data
# Now we move on to the testing part:

test_features = test_df[feature_cols].to_numpy()
case_ids = test_df['Case#'].values
sp = test_df['Spacecraft#'].values
results = []        # helps generate the final csv file


# decoders
loc_map = {0: "BP1", 1: "BP2", 2: "BP3", 3: "BP4", 4: "BP5", 5: "BP6", 6: "BP7", 7: "BV1"}
sv_map = {0: "SV1", 1: "SV2", 2: "SV3", 3: "SV4"}

for i in range(len(test_features)):
    row = test_features[i].reshape(1, -1)
    c_id = case_ids[i]
    sp_no = sp[i]

    # Assuming Normal Initally:
    t1, t2, t3, t4, t5 = 0, 0, 0, 0, 100
    condition_text = "Normal"
    pred_t1 = model_t1.predict(row)[0]
    
    # 1a. Is it unknown or abnormal?
    if pred_t1 == 1: 
        # Either Unknown or Abnormal 
        t1 = 1
        gate_score = gatekeeper.predict(row)[0]
        
        if gate_score == -1:
            # Unknown anomaly identified
            t2 = 1  # Encoding based on ground truth
            condition_text = "Unknown anomaly"
        else:
            # Either bubble anomaly or valve fault
            pred_t2 = model_t2.predict(row)[0]
            
            if pred_t2 == 0: 
                # Bubble anomaly idenitified
                t2 = 2  # Encoding based on ground truth
                
                # To find the bubble anomaly location
                pred_t3 = model_t3.predict(row)[0]
                t3 = pred_t3 + 1 # Shift to 1-8 as BP1=1, etc to match ground truth
                
                condition_text = f"{loc_map[pred_t3]} bubble anomaly"
                
            else:
                # Valve fault identified
                t2 = 3  # Encoding based on ground truth
                
                # To find the valve fault location
                pred_t4 = model_t4.predict(row)[0] 
                t4 = pred_t4 + 1 # Shift to 1-4 SV1=1, etc. to match ground truth
                
                # To find the opening ratio
                pred_t5 = model_t5.predict(row)[0]
                t5 = int(max(0, min(100, pred_t5)))
                
                condition_text = f"{sv_map[pred_t4]} valve fault"


    results.append([sp_no, c_id, t1, t2, t3, t4, t5, condition_text])


# Making the dataframe: 

cols = ['Spacecraft No.', 'ID', 'task1', 'task2', 'task3', 'task4', 'task5', 'Test condition']
submission_df = pd.DataFrame(results, columns=cols)  

# Saving into a csv file:
submission_df.to_csv("Submission.csv", index=False)
